In [ ]:
from urllib.request import urlopen
import ssl
import json

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

dataTrain = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_001.json", context=ctx).read().decode()
dataTest = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/test/dialogues_001.json", context=ctx).read().decode()

myJsonTrain = json.loads(dataTrain)
myJsonTest = json.loads(dataTest)
print (json.dumps(myJsonTrain[0], indent=1)[0:100])
print (json.dumps(myJsonTest[0], indent=1)[0:100])



{
 "dialogue_id": "PMUL4398.json",
 "services": [
  "restaurant",
  "hotel"
 ],
 "turns": [
  {
   "
{
 "dialogue_id": "MUL0484.json",
 "services": [
  "attraction",
  "train"
 ],
 "turns": [
  {
   "f


In [ ]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.5.0


In [ ]:
import re

from nltk.util import ngrams

timepat = re.compile("\d{1,2}[:]\d{1,2}")
pricepat = re.compile("\d{1,3}[.]\d{1,2}")

fin = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/utils/mapping.pair", context=ctx)
replacements = []
for line in fin.readlines():
    line = line.decode().replace('\n', '')
    tok_from, tok_to = line.replace('\n', '').split('\t')
    replacements.append((' ' + tok_from + ' ', ' ' + tok_to + ' '))


def insertSpace(token, text):
    sidx = 0
    while True:
        sidx = text.find(token, sidx)
        if sidx == -1:
            break
        if sidx + 1 < len(text) and re.match('[0-9]', text[sidx - 1]) and \
                re.match('[0-9]', text[sidx + 1]):
            sidx += 1
            continue
        if text[sidx - 1] != ' ':
            text = text[:sidx] + ' ' + text[sidx:]
            sidx += 1
        if sidx + len(token) < len(text) and text[sidx + len(token)] != ' ':
            text = text[:sidx + 1] + ' ' + text[sidx + 1:]
        sidx += 1
    return text


def normalize(text):
    # lower case every word
    text = text.lower()

    # replace white spaces in front and end
    text = re.sub(r'^\s*|\s*$', '', text)

    # hotel domain pfb30
    text = re.sub(r"b&b", "bed and breakfast", text)
    text = re.sub(r"b and b", "bed and breakfast", text)

    # normalize phone number
    ms = re.findall('\(?(\d{3})\)?[-.\s]?(\d{3})[-.\s]?(\d{4,5})', text)
    if ms:
        sidx = 0
        for m in ms:
            sidx = text.find(m[0], sidx)
            if text[sidx - 1] == '(':
                sidx -= 1
            eidx = text.find(m[-1], sidx) + len(m[-1])
            text = text.replace(text[sidx:eidx], ''.join(m))

    # normalize postcode
    ms = re.findall('([a-z]{1}[\. ]?[a-z]{1}[\. ]?\d{1,2}[, ]+\d{1}[\. ]?[a-z]{1}[\. ]?[a-z]{1}|[a-z]{2}\d{2}[a-z]{2})',
                    text)
    if ms:
        sidx = 0
        for m in ms:
            sidx = text.find(m, sidx)
            eidx = sidx + len(m)
            text = text[:sidx] + re.sub('[,\. ]', '', m) + text[eidx:]

    # weird unicode bug
    text = re.sub(u"(\u2018|\u2019)", "'", text)

    # replace time and and price
    times = re.findall(timepat, text)
    prices = re.findall(pricepat, text)
    text = re.sub(timepat, ' [value_time] ', text)
    text = re.sub(pricepat, ' [value_price] ', text)
    #text = re.sub(pricepat2, '[value_price]', text)

    # replace st.
    text = text.replace(';', ',')
    text = re.sub('$\/', '', text)
    text = text.replace('/', ' and ')

    # replace other special characters
    text = text.replace('-', ' ')
    text = re.sub('[\":\<>@\(\)]', '', text)

    # insert white space before and after tokens:
    for token in ['?', '.', ',', '!']:
        text = insertSpace(token, text)

    # insert white space for 's
    text = insertSpace('\'s', text)

    # replace it's, does't, you'd ... etc
    text = re.sub('^\'', '', text)
    text = re.sub('\'$', '', text)
    text = re.sub('\'\s', ' ', text)
    text = re.sub('\s\'', ' ', text)
    for fromx, tox in replacements:
        text = ' ' + text + ' '
        text = text.replace(fromx, tox)[1:-1]

    # remove multiple spaces
    text = re.sub(' +', ' ', text)

    # concatenate numbers
    tmp = text
    tokens = text.split()
    i = 1
    while i < len(tokens):
        if re.match(u'^\d+$', tokens[i]) and \
                re.match(u'\d+$', tokens[i - 1]):
            tokens[i - 1] += tokens[i]
            del tokens[i]
        else:
            i += 1
    text = ' '.join(tokens)

    for time in times:
      text = re.sub('\\[value_time\\]', time, text, 1)

    for price in prices:
      text = re.sub('\\[value_price\\]', price, text, 1)

    return text

In [ ]:
# filter
import string
labels = set()

def filter(data):
  global labels

  result = []

  for i, obj in enumerate(data):
      turns = obj.get("turns", None)
      for turn in turns:
          if turn:
              utterance = turn.get("utterance", None)
              if utterance:
                  frames = turn.get("frames", None)
                  if frames:
                      dic = dict()
                      res = [[], []]
                      for frame in frames: 
                          state = frame.get("state", None)
                          service = frame.get("service", None)
                          if service and service == "restaurant" and state:
                              slot_values = state.get("slot_values", None)
                              if slot_values and len(slot_values) != 0:
                                  for key, value in slot_values.items():
                                      for elem in value:
                                        dic[elem] = key

                      text = normalize(utterance)
                      for word in text.split(" "):
                          res[0].append(word)
                          res[1].append(dic.get(word, "O"))
                          labels.add(dic.get(word, "O"))    
                      
                      result.append(res)




  return result

train_dataset = filter(myJsonTrain)
for i, elem in enumerate(train_dataset):
      # print(elem)
      if i > 30:
        # print("--------------------")
        break
test_dataset = filter(myJsonTest)
for i, elem in enumerate(test_dataset):
      # print(elem)
      if i > 30:
        # print("--------------------")
        break

print(labels)
test_dataset[6:9]

{'restaurant-food', 'restaurant-booktime', 'O', 'restaurant-area', 'restaurant-bookpeople', 'restaurant-bookday', 'restaurant-name', 'restaurant-pricerange'}


[[['can',
   'you',
   'book',
   'me',
   'a',
   'table',
   'for',
   '11:00',
   'on',
   'friday',
   '?'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'restaurant-booktime',
   'O',
   'restaurant-bookday',
   'O']],
 [['actually', ',', 'for', '4', ',', 'please', '.'],
  ['O', 'O', 'O', 'restaurant-bookpeople', 'O', 'O', 'O']],
 [['great',
   ',',
   'can',
   'you',
   'also',
   'get',
   'me',
   'information',
   'or',
   'architecture',
   'in',
   'the',
   'area'],
  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]]

In [ ]:
for i, label in enumerate(train_dataset[1]):
    if 'other' in label:
      train_dataset[1][i] = 'O'

for i, label in enumerate(test_dataset[1]):
    if 'other' in label:
      test_dataset[1][i] = 'O'
      
if 'other' in labels: labels.remove('other')
labels.add('O')
print(labels)

# create character vocab
all_text = " ".join([" ".join(x[0]) for x in train_dataset+test_dataset])
vocab = sorted(set(all_text))

# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)

print(idx2label)
print(char2idx)

{'restaurant-food', 'restaurant-booktime', 'O', 'restaurant-area', 'restaurant-bookpeople', 'restaurant-bookday', 'restaurant-name', 'restaurant-pricerange'}
{'restaurant-food', 'restaurant-booktime', 'O', 'restaurant-area', 'restaurant-bookpeople', 'restaurant-bookday', 'restaurant-name', 'restaurant-pricerange'}
{' ': 1, '!': 2, '#': 3, '$': 4, '&': 5, "'": 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, '`': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49}


In [ ]:
def split_char_labels(eg):
  '''
  For a given input/output example, break tokens into characters while keeping 
  the same label.
  '''

  tokens = eg[0]
  labels = eg[1]

  input_chars = []
  output_char_labels = []

  for token,label in zip(tokens,labels):
    input_chars.extend([char for char in token])
    input_chars.extend(' ')
    output_char_labels.extend([label]*len(token))
    output_char_labels.extend('O')

  return [[char2idx[x] for x in input_chars[:-1]], np.array([label2idx[x] for x in output_char_labels[:-1]])]


train_formatted = [split_char_labels(eg) for eg in train_dataset]
test_formatted = [split_char_labels(eg) for eg in test_dataset]

print(len(train_formatted))
print(len(test_formatted))

3907
4242


In [ ]:
# training generator
def gen_train_series():

    for eg in train_formatted:
      yield eg[0],eg[1]

# validation generator
def gen_valid_series():

    for eg in valid_formatted:
      yield eg[0],eg[1]

# test generator
def gen_test_series():

  for eg in test_formatted:
      yield eg[0],eg[1]
  
# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000

# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch_test.take(1):
  print(input_example_batch)
  print(target_example_batch)

tf.Tensor(
[[32  1 37 ...  0  0  0]
 [32  1 46 ...  0  0  0]
 [25 28 29 ...  0  0  0]
 ...
 [26 24 37 ...  0  0  0]
 [43 31 24 ...  0  0  0]
 [32  1 46 ...  0  0  0]], shape=(128, 154), dtype=int32)
tf.Tensor(
[[3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 ...
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 [3 3 3 ... 0 0 0]], shape=(128, 154), dtype=int32)


In [ ]:
vocab_size = len(vocab)+1

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

label_size = len(labels)  

# build LSTM model
def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
      model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                    batch_input_shape=[batch_size, None],mask_zero=True),
          tf.keras.layers.LSTM(rnn_units,
                               return_sequences=True,
                               stateful=True,
                               recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(label_size)
          ])
      return model

model = build_model(
        vocab_size=len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          12800     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 9)            9225      
Total params: 5,269,001
Trainable params: 5,269,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import os

# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss,metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [11]:
EPOCHS=20
history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_test,callbacks=[checkpoint_callback])

Epoch 1/20
30/30 [==============================] - 774s 26s/step - loss: 0.1277 - sparse_categorical_accuracy: 0.9439 - val_loss: 0.0588 - val_sparse_categorical_accuracy: 0.9772
Epoch 2/20
30/30 [==============================] - 750s 25s/step - loss: 0.0552 - sparse_categorical_accuracy: 0.9775 - val_loss: 0.0565 - val_sparse_categorical_accuracy: 0.9772
Epoch 3/20
30/30 [==============================] - 747s 25s/step - loss: 0.0524 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.0535 - val_sparse_categorical_accuracy: 0.9772
Epoch 4/20
30/30 [==============================] - 735s 25s/step - loss: 0.0514 - sparse_categorical_accuracy: 0.9776 - val_loss: 0.0515 - val_sparse_categorical_accuracy: 0.9772
Epoch 5/20
30/30 [==============================] - 752s 25s/step - loss: 0.0491 - sparse_categorical_accuracy: 0.9773 - val_loss: 0.0498 - val_sparse_categorical_accuracy: 0.9772
Epoch 6/20
30/30 [==============================] - 744s 25s/step - loss: 0.0483 - sparse_categorica

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

  pred=model.predict_on_batch(input_example_batch)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()

  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

[[     0      0      0      0      0      0      0      0]
 [     0      6      0      0      0      0      0      0]
 [  1571    900 268085    840    181   1137    187   1329]
 [     0      0      0      0      0      0      0      0]
 [     0      0      0      0      0      0      0      0]
 [     0      0     17      0      0     78      0      0]
 [     0      0      0      0      0      0      0      0]
 [     1      0     21      0      0      0      0     17]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         0
         2.0       0.01      1.00      0.01         6
         3.0       1.00      0.98      0.99    274230
         4.0       0.00      0.00      0.00         0
         5.0       0.00      0.00      0.00         0
         6.0       0.06      0.82      0.12        95
         7.0       0.00      0.00      0.00         0
         8.0       0.01      0.44      0.02        39

    accuracy                           0.98    274370
   macro avg       0.14      0.40      0.14    274370
weighted avg       1.00      0.98      0.99    274370

